### LCEL Deepdive

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
prompt = ChatPromptTemplate.from_template("Explain about {topic} in detail")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Ice cream is a frozen dessert that is typically made from a mixture of cream, sugar, and flavorings. It can come in a variety of flavors, including vanilla, chocolate, strawberry, and mint chocolate chip, among many others. Ice cream is a popular treat around the world and is enjoyed by people of all ages.\n\nThe basic ingredients in ice cream include cream, sugar, and flavorings. Cream is the key ingredient that gives ice cream its smooth and creamy texture. Sugar is added to sweeten the dessert, while flavorings such as vanilla extract, cocoa powder, or fruit puree are used to give ice cream its distinct taste.\n\nThe process of making ice cream involves mixing together the ingredients and then freezing the mixture until it becomes solid. This can be done in a traditional ice cream maker, which churns the mixture as it freezes, or in a more modern freezer that uses a compressor to freeze the mixture.\n\nThere are also many variations of ice cream, such as gelato, sorbet, and frozen 

In [32]:
print(prompt.invoke({"topic": "ice cream"}))

messages=[HumanMessage(content='Explain about ice cream in detail', additional_kwargs={}, response_metadata={})]


In [33]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

AIMessage(content='Why did the ice cream truck break down?\nBecause it had too many "scoops!"', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 15, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CNqlGrP8YIjdaOjMxv7uCdspECjg8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--03d2ec42-4785-40b6-bf6b-b579740acfac-0', usage_metadata={'input_tokens': 15, 'output_tokens': 19, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### What is this "|" in Python?

In [34]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        return data

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)



In [35]:
class AddTen(CRunnable):
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"

In [36]:
a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

In [37]:
result = chain.invoke(10)
print(result)

AddTen:  10
Multiply by 2:  20
Convert to string:  40
Result: 40


### Runnables from LangChain

In [38]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [39]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [40]:
def input_to_upper(input: str):
    output = input.upper()
    return output

In [41]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

In [42]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [43]:
chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [44]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'},
 'y': {'input': 'hello', 'input2': 'goodbye'}}

In [45]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

In [46]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'y': 'goodbye'}

### Nested chains - now it gets more complicated!

In [47]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

In [48]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

In [49]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': 'HELLO', 'y': 'goodbye'}

In [50]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(input):
    return 100

def multiply(input):
    return input * 10

In [51]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}}

In [52]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

In [53]:
result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'extra': 100}


### Combine multiple chains (incl. coercion)

In [54]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

In [55]:
new_chain.invoke({"extra": "test"})

'TEST'

In [56]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})

'100'

### Real Work example

In [59]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


vectorstore = FAISS.from_texts(
    ["Cats love thuna"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

In [60]:
rag_chain.invoke("What do cats like to eat?")

'Tuna'

In [61]:
RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()}).invoke("What do cats like to eat?")

{'context': 'Cats love thuna', 'question': 'What do cats like to eat?'}

In [62]:
prompt.invoke({"context": "Cats love thuna", "question": "What do cats like to eat?"})

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\nCats love thuna\n\nQuestion: What do cats like to eat?\n', additional_kwargs={}, response_metadata={})])

In [64]:
ChatOpenAI().invoke(prompt.invoke({"context": "Cats love thuna", "question": "What do cats like to eat?"}))

AIMessage(content='Tuna', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 32, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CNqrflqjAhiHN5DpDapxgvtK2FDcv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--859fa1df-80af-4182-afbb-698a4149c858-0', usage_metadata={'input_tokens': 32, 'output_tokens': 2, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})